In [ ]:
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pwd

/content/drive/MyDrive


In [ ]:
from dlc_practical_prologue import *

In [ ]:
train_input, train_target, test_input, test_target=load_data(cifar=True,one_hot_labels=True,normalize=True,flatten=True) 

* Using CIFAR
Files already downloaded and verified
Files already downloaded and verified
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [ ]:
train_input.shape, train_target.shape

(torch.Size([1000, 3072]), torch.Size([1000, 10]))

In [ ]:
train_input[0].shape

torch.Size([3072])

In [ ]:
from torchvision import datasets, transforms

#mnist_train = datasets.MNIST("data", train=True, download=True, transform=transforms.ToTensor())

#mnist_test = datasets.MNIST("data", train=False, download=True, transform=transforms.ToTensor())

In [ ]:
#print(mnist_train)
#print(mnist_test)

In [ ]:
#mnist_train[0][0].shape

In [ ]:
#a = torch.flatten(mnist_train[0][0])
#a.shape

#1)Let us write the activation function et his derivative

In [ ]:

def sigma(x):
  tanh=torch.tanh(x)
  return tanh

In [ ]:
def dsigma(x):
  dtanh= 1- torch.pow(torch.tanh(x),2)
  return dtanh

#2)Let us write the function loss


In [ ]:
def loss(v,t):
  loss=torch.norm(v - t)
  return torch.pow(loss,2)

In [ ]:
def dloss(v,t):
  return 2*(v - t)

In [ ]:
#def dloss(v,t):
  #v=v.requires_grad(v)
  #l=loss(v,t)
  #l.backward()
  #return v.grad

#3Let us write the function for forward and backward
#********forward


In [ ]:
def forward_pass(w1, b1, w2, b2, x):
  x0=x
  s1=torch.mm(w1,x0) + b1
  x1= sigma(s1)
  s2=torch.mm(w2,x1) + b2
  x2=sigma(s2)
  return x0, s1, x1,s2,x2



#*******backward

In [ ]:
def backward_pass(w1,b1,w2,b2,t,x, s1, x1, s2, x2,dl_dw1, dl_db1, dl_dw2, dl_db2):
  #print(x.shape,s1.shape,w1.shape, b1.shape)
  dl_dx2 = dloss(x2, t)
  dx2_ds2 = dsigma(s2)
  #print('dl_dx2:  ', dl_dx2.shape)
  dl_ds2 = torch.mul(dl_dx2, dx2_ds2)
  #print('dx2_ds2:  ', dx2_ds2.shape)
  dl_dw2 += torch.mm(dl_ds2, torch.t(x1))
  dl_db2 += dl_ds2
  dl_dx1 = torch.mm(torch.t(w2), dl_ds2)
  dx1_ds1 = dsigma(s1)
  #print('dx1_ds1:  ', dx1_ds1.shape)
  dl_ds1 = torch.mul(dl_dx1, dx1_ds1)
  dl_dw1 += torch.mm(dl_ds1, torch.t(x))
  dl_db1 += dl_ds1
  return dl_dw1, dl_db1, dl_dw2, dl_db2
 

In [ ]:
def error(test_set, test_target_set, w1, b1, w2, b2):
    n = test_set.shape[0]
    num_errors = 0
    for i in range(n):
        x = test_set[i].resize(test_set[i].shape[0], 1)
        t = torch.argmax(test_target_set[i])
        _, _, _, _, predicted = forward_pass(w1, b1, w2, b2, x)
        predicted_class = torch.argmax(predicted, 0)
        if t != predicted_class:
            num_errors += 1
    return num_errors/n

#4)Train the network

In [ ]:
def train():
#4)1)let us load the data using the provided prologue.load˙data function, with one-hot label vectors
#and normalized inputs. Multiply the target label vectors by ζ = 0.9 (so that they are strictly in
#the value range of tanh
  train_input, train_target, test_input, test_target= load_data(cifar=True, one_hot_labels=True,normalize=True, flatten=True)

  test_target = test_target.type(torch.LongTensor)*0.9
  train_target = train_target.type(torch.LongTensor)*0.9
  eps = 0.0000001
  print(eps)
#4)2)let us create the four weight and bias tensors, and fill them with random values sampled according to N(0,eps ) with  eps= 1e − 6.
  
  w1 = torch.zeros(50, 3072).normal_(0, eps)
  b1 = torch.zeros(50, 1).normal_(0, eps)
  w2 = torch.zeros(10, 50).normal_(0, eps)
  b2 = torch.zeros(10, 1).normal_(0, eps)
#4)3) let create the four tensors to sum up the gradients on individual samples, with respect to the weights and biases
  eta = 0.1/train_input.shape[0]
  for i in range(1000):
    dl_dw1 = torch.zeros(50, 3072)
    dl_db1 = torch.zeros(50, 1)
    dl_dw2 = torch.zeros(10, 50)
    dl_db2 = torch.zeros(10, 1)
#4)3)4
    for j in range(1000):
      x = train_input[j].resize_(3072, 1)
      t = train_target[j,:].reshape(-1,1)
      x0, s1, x1, s2, x2 = forward_pass(w1, b1, w2, b2, x)
      backward_pass(w1, b1, w2, b2, t, x, s1, x1, s2, x2, dl_dw1, dl_db1, dl_dw2, dl_db2)
    w1 = w1 - eta * dl_dw1
    w2 = w2 - eta * dl_dw2
    b1 = b1 - eta * dl_db1
    b2 = b2 - eta * dl_db2
  print("Training error: ", error(train_input, train_target, w1, b1, w2, b2))
  print("Test error: ", error(test_input, test_target, w1, b1, w2, b2))
  

In [ ]:
train()

* Using CIFAR
Files already downloaded and verified
Files already downloaded and verified
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples
1e-07
Training error:  0.062
Test error:  0.742


/usr/local/lib/python3.7/dist-packages/torch/tensor.py:474: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
